In [1]:
import sys
import os

# Go up one level to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [ ]:
# Imports
import torch

# Set random seeds for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [31]:
from types import SimpleNamespace
dataset = SimpleNamespace()
dataset.L, dataset.H = 567, 96

In [ ]:
log_dir = f'../tb_test/{dataset.L}_{dataset.H}'

In [34]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=log_dir)


In [38]:
writer.add_hparams({"model": 'LinearPFN2', "l": 12},{"ET/mae": 16, "ET/mse": 25, "EX/mae": 86, "EX/mse": 15})
writer.add_hparams({"model": 'TabPFN2', "l": 11},{"ET/mae": 15, "ET/mse": 56, "EX/mae": 45, "EX/mse": 48})

In [37]:
writer.close()

In [ ]:
writer.add_hparams({"model": 'DPFN', "l": 8},{"E/mae": 12, "ET/mse": 66, "EX/mae": 45, "EX/mse": 34})


In [ ]:
import json
def log_hparam_run(model_name, w, hparams: dict, metrics: dict):

    # optional: store raw payload for provenance
    # with open(os.path.join(run_dir, "hparams.json"), "w") as f:
    #     json.dump({"hparams": hparams, "metrics": metrics}, f, indent=2)

    # 1) add_hparams logs both the session and the metrics for HParams tab
    w.add_hparams(
        hparam_dict={
            "model": model_name,              # categorical axis
            # "dataset": dataset_name,          # categorical axis
            **{k: v for k, v in hparams.items()  # only scalars/strings here
               if isinstance(v, (int, float, str, bool)) or v is None}
        },
        metric_dict={
            # "test/mse": float(metrics["mse"]),
            "test/mce": float(metrics["mce"]),
            # add anything else you want as numeric axes
            # **{k: float(v) for k, v in metrics.items()
            #    if k not in ("mse","mae") and isinstance(v, (int, float))}
        }
    )
    # 2) also write them as regular scalars at step 0 so they appear elsewhere
    # w.add_scalar("test/mse", float(metrics["mse"]), 0)
    # w.add_scalar("test/mae", float(metrics["mae"]), 0)
    w.close()

m = {
    "LinearPFN": {
        "ETTh1": {"mae": 0.5, "mse": 0.5}, 
        "ETTh2": {"mae": 0.5, "mse": 0.5}, 
    },
    "TabPFNv2": {
        "ETTh1": {"mae": 0.7, "mse": 0.7}, 
        "ETTh2": {"mae": 0.7, "mse": 0.7}, 
    },
}
for model_name in ["LinearPFN", "TabPFNv2"]:
    for ds in ["ETTh1", "ETTh2"]:
        m = {"mce": 0.8}
        hp = {
            "d_model": 5,
            "L_blk": 7,
            "H": 8,
            "horizon": 9,
        }
        log_hparam_run(model_name, writer, hp, m)

In [39]:
import os
path = "../ckpts/meta/" 
dir_list = os.listdir(path)
dir_list

['LinearPFN_L512_H96_d256_Lblk6_n8_dff1024_do0.1.json']

In [42]:
with open(os.path.join(path, dir_list[0])) as f:
    d = json.load(f)
    print(d)

{'dataset_meta': 'C32-1536_Q1-32', 'tag': 'synth_v2', 'hparams': {'L': 512, 'H': 96, 'dropout': 0.1, 'd_model': 256, 'd_ff': 1024, 'L_blk': 6, 'n_heads': 8}}


In [49]:
model = 'LinearPFNv2'
data_version = '0'
seq, pred = 96, 96
d_model = 256
n_heads = 8
e_layers = 6
d_ff = 1024

In [79]:
hp_dict = {
    'model': model,
    'data_version': data_version,
    'd_model': d_model, 'n_heads': n_heads, 'L': e_layers, 'd_ff': d_ff
}

In [52]:
csv_path = f'{model}/v{data_version}/sl{seq}_pl{pred}_dm{d_model}_nh{n_heads}_el{6}_df{d_ff}'

In [77]:
import csv

metric_d = dict()

with open(f'../../results/{csv_path}/results.csv', mode='r', newline='') as file:
    csv_dict_reader = csv.DictReader(file)
    for row in csv_dict_reader:
        # print(row['column_name']) # Access data by column name
        if row['train_budget'] == '0.25': continue
        print(row) # Prints the entire row as an OrderedDict
        metric_d[f'{row["dataset"]}/mse'] = row['mse']
        metric_d[f'{row["dataset"]}/mae'] = row['mae']

{'dataset': 'electricity', 'train_budget': '0.75', 'seq_len': '96', 'pred_len': '96', 'd_model': '256', 'e_layers': '6', 'n_heads': '8', 'd_ff': '1024', 'mae': '0.40560937', 'mse': '0.32773888', 'rmse': '0.57248485', 'mape': '1.9589453', 'mspe': '698.134'}
{'dataset': 'ETTh1', 'train_budget': '0.75', 'seq_len': '96', 'pred_len': '96', 'd_model': '256', 'e_layers': '6', 'n_heads': '8', 'd_ff': '1024', 'mae': '0.29835692', 'mse': '0.13832837', 'rmse': '0.37192523', 'mape': '0.21985614', 'mspe': '0.07311442'}
{'dataset': 'ETTh2', 'train_budget': '0.75', 'seq_len': '96', 'pred_len': '96', 'd_model': '256', 'e_layers': '6', 'n_heads': '8', 'd_ff': '1024', 'mae': '0.44656366', 'mse': '0.3379732', 'rmse': '0.5813546', 'mape': '0.95889777', 'mspe': '43.865482'}
{'dataset': 'ETTm1', 'train_budget': '0.75', 'seq_len': '96', 'pred_len': '96', 'd_model': '256', 'e_layers': '6', 'n_heads': '8', 'd_ff': '1024', 'mae': '0.175294', 'mse': '0.05335857', 'rmse': '0.23099475', 'mape': '0.13317616', 'mspe

In [78]:
metric_d

{'electricity/mse': '0.32773888',
 'electricity/mae': '0.40560937',
 'ETTh1/mse': '0.13832837',
 'ETTh1/mae': '0.29835692',
 'ETTh2/mse': '0.3379732',
 'ETTh2/mae': '0.44656366',
 'ETTm1/mse': '0.05335857',
 'ETTm1/mae': '0.175294',
 'ETTm2/mse': '0.109666035',
 'ETTm2/mae': '0.24798533',
 'exchange/mse': '0.3684906',
 'exchange/mae': '0.4340186',
 'traffic/mse': '0.30715355',
 'traffic/mae': '0.40437207',
 'weather/mse': '0.0016890195',
 'weather/mae': '0.031717733'}

In [ ]:

log_dir = f'../tb_test/{seq}_{pred}'
writer = SummaryWriter(log_dir=log_dir)
writer.add_hparams(hparam_dict=hp_dict, metric_dict=metric_d)
writer.close()

In [44]:
d['hparams']

{'L': 512,
 'H': 96,
 'dropout': 0.1,
 'd_model': 256,
 'd_ff': 1024,
 'L_blk': 6,
 'n_heads': 8}

In [47]:
os.path.splitext(dir_list[0])[0]

'LinearPFN_L512_H96_d256_Lblk6_n8_dff1024_do0.1'